In [11]:
!pip install langchain langchain_community


  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.0 MB/s eta 0:00:00
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [langchain_community]ngchain_community]


In [40]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama
import json

In [42]:
#Step 1: Load your raw prompt from the .txt file
with open("/Users/sevios/ai_projects/business_comp_ai_agent/prompts/compatibility_prompt.text", "r") as f:
    raw_prompt = f.read()

In [43]:
#Step 2: Create a PromptTemplate with variable names matching your placeholders
prompt = PromptTemplate(
    template=raw_prompt,
    input_variables=["business_a", "business_b"]
)



In [44]:
#print the formatted prompt with dummy data to check if it works
print(prompt.format(
    business_a= "A seafood distributor operating in coastal Ghana with 5 refrigerated trucks monthly capacity of 15 tons.",
    business_b="A restaurant chain in Accra seeking weekly delivery of 3 tons of frozen tilapia, fully certified."
))

You are a business compatibility analyst evaluating whether two businesses should form a commercial partnership.

Use the following format for every analysis:

---
EXAMPLE
BUSINESS A:
A wholesale coffee supplier based in Kenya with 10-ton monthly roasting capacity. Certified Organic and Rainforest Alliance certified.

BUSINESS B: 
A European café chain sourcing 5 tons/month of single-origin coffee, seeking certified organic partners.

COMPATABILITY: 

With 15 tons capacity, business A can meet business B's demand. Both businesses also share the same certifications and values with both having (certified organic, hccp).

Verdict: Compatible

Reasoning:
- Product/service fit: Business A supplies frozen tilapia, matching the exact seafood type required by Business B.
- Capacity/logistics: Business A’s monthly capacity of 15 tons covers Business B’s weekly demand of 3 tons (≈12 tons/month), with refrigerated transport ensuring freshness.
- Region/logistical feasibility: Both businesses oper

In [45]:
#Connect to Ollama running gemma
llm = Ollama(model="gemma:2b")

#Build the chain
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

In [24]:
#Run the agent
response = chain.run({
    "business_a": "A seafood distributor operating in coastal Ghana with 5 refrigerated trucks and monthly capacity of 15 tons.",
    "business_b": "A restaurant chain in Accra seeking weekly delivery of 3 tons of frozen tilapia, fully certified."
})



> Entering new LLMChain chain...
Prompt after formatting:
You are a business compatability evalutation agent.

Your task is to determine if Business A and Business B are compatible as partners.

Evaluate based on:
- Product or service fit
- Capacity and delivery alignment
- Region or logistical feasability
- shared certifications or values

BUSINESS A:
A seafood distributor operating in coastal Ghana with 5 refrigerated trucks and monthly capacity of 15 tons.

BUSINESS B:
A restaurant chain in Accra seeking weekly delivery of 3 tons of frozen tilapia, fully certified.

Provide:
- A verdict: "Compatable" or "Not Compatible"
- For each evaluation area (fit, delivery, logistics, certifications), provide 1-2 sentences for reasoning.
- When possible, include specific details from the profiles (e.g., "certified organic", "monthly capacity of 15 tons", "Accra region").

> Finished chain.


In [46]:
def build_profile(business: dict) -> str:
    lines = [
        f"{business['name']} is a {business['type']} located in {business['location']}.",
    ]

    if "capacity" in business:
        lines.append(f"Monthly capacity: {business['capacity']}.")
    if "transport" in business:
        lines.append(f"Transport assets: {business['transport']}.")
    if "demand" in business:
        lines.append(f"Demand requirement: {business['demand']}.")
    if "certifications" in business:
        lines.append(f"Certifications {','.join(business['certifications'])}.")
    if "values" in business:
        lines.append(f"Company values: {', '.join(business['values'])}.")
    if "notes" in business:
        lines.append(business["notes"])

    return " ".join(lines)

In [ ]:
def evaluate_business_pairs(business_a: dict, business_b: dict) -> str:
    with open("prompts/compatibility_prompt.txt", "r") as f:
        prompt_template_str = f.read()

    prompt = PromptTemplate.from_template(prompt_template_str)
    llm = Ollama(model="gemma:2b")  # adjust if you’re using another model

    chain = LLMChain(llm=llm, prompt=prompt)

    profile_a = build_profile(business_a)
    profile_b = build_profile(business_b)

    response = chain.run({
        "business_a": profile_a,
        "business_b": profile_b
    })

    return response

In [ ]:
def evaluate_parties():
    ...

In [47]:
with open("/Users/sevios/ai_projects/business_comp_ai_agent/data/business_profiles.json", "r") as f:
    data = json.load(f)
    business_a = build_profile(data["businesses"][0])
    business_b = build_profile(data["businesses"][1])


In [48]:
response = chain.run({
    "business_a": business_a,
    "business_b": business_b
})

print(response)



> Entering new LLMChain chain...
Prompt after formatting:
You are a business compatibility analyst evaluating whether two businesses should form a commercial partnership.

Use the following format for every analysis:

---
EXAMPLE
BUSINESS A:
A wholesale coffee supplier based in Kenya with 10-ton monthly roasting capacity. Certified Organic and Rainforest Alliance certified.

BUSINESS B: 
A European café chain sourcing 5 tons/month of single-origin coffee, seeking certified organic partners.

COMPATABILITY: 

With 15 tons capacity, business A can meet business B's demand. Both businesses also share the same certifications and values with both having (certified organic, hccp).

Verdict: Compatible

Reasoning:
- Product/service fit: Business A supplies frozen tilapia, matching the exact seafood type required by Business B.
- Capacity/logistics: Business A’s monthly capacity of 15 tons covers Business B’s weekly demand of 3 tons (≈12 tons/month), with refrigerated transport ensuring fres